**Use of ResNet18 on pre-trained is False**
- ImageFolder('path_to_data', 'transform_instance'): to load the images from folder having seperate folder for each class
- transform.compose(): pass list of Augmentation tech. 
- Normalization: Code is written for to find out data std and mean
- DataLoade('imagefolder_instance', batch_size, shuffle) 
- Training on smaller dataset using random_split(dataset, [training_size, test_size]).
- 


In [1]:
import numpy as np
import torch
import os
from torch.utils.data import DataLoader
import pandas as pd
import matplotlib.pyplot as plt
import PIL
import torchvision.transforms as transform
from torchvision.datasets import ImageFolder
import time
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [2]:
import os
path = 'NewPlantDiease/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)'
train_data1 = path + '/train'
val_data1 = path + '/valid'
diease = os.listdir(train_data1)

In [3]:
# train_data, testing_data  = train_test_split(train_data1, test_size = 0.2, random_state=42)

In [4]:
# mean = [0.4757, 0.5001, 0.4264]
# std = [0.2165, 0.1957, 0.2320]
# transforms = transform.Compose([transform.Resize((256, 256)), transform.ToTensor(), transform.Normalize(mean=mean, std=std)])

In [5]:
# train_datasets = ImageFolder(root = train_data1, transform=transforms)
# LEN_TRAIN = len(train_datasets)
# # LEN_val = len(val_datasets)
# print(LEN_TRAIN)

In [6]:
# # train_data, testing_data  = train_test_split(train_data1, test_size = 0.3, random_state=42)
# # len(train_data), len(testing_data)
# train_data = int(0.9 * LEN_TRAIN)
# testing_data = LEN_TRAIN - train_data
# train_set, val_set = torch.utils.data.random_split(train_datasets, [train_data, testing_data])

In [7]:

# validation_set = val_set.dataset
# len(validation_set.classes),len(val_set)

In [8]:
# Train_dataloader = DataLoader(dataset = val_set,  batch_size= 256, shuffle = True )


In [9]:
# val_datasets = ImageFolder(root = val_data1, transform=transforms)
# train_data1 = int(0.8 * len(val_datasets))
# testing_data1 = len(val_datasets) - train_data1
# train_set1, val_set1 = torch.utils.data.random_split(val_datasets, [train_data1, testing_data1])
# Val_dataloader = DataLoader(dataset = val_set1,  batch_size= 256, shuffle = False )
# validation_set1 = Val_dataloader.dataset
# len(validation_set.classes),len(val_set1)


In [10]:
mean = [0.4757, 0.5001, 0.4264]
std = [0.2165, 0.1957, 0.2320]
transforms = transform.Compose([transform.Resize((256, 256)), transform.ToTensor(), transform.Normalize(mean=mean, std=std)])
train_datasets = ImageFolder(root = train_data1, transform=transforms)
val_datasets = ImageFolder(root = val_data1, transform=transforms)
Train_dataloader = DataLoader(dataset = train_datasets,  batch_size= 256, shuffle = True )
Val_dataloader = DataLoader(dataset = val_datasets,  batch_size= 256, shuffle = False )

In [11]:
LEN_TRAIN = len(train_datasets)
LEN_val = len(val_datasets)
print(LEN_TRAIN, LEN_val)

70295 17572


In [12]:
# LEN_TRAIN = len(testing_data)
# # LEN_val = len(val_datasets)
# print(LEN_TRAIN)

In [13]:
# images, labels = next(iter(testing_data))
# print(images.size())
# # Access the first image and label
# first_image = images[2]
# first_label = labels[0]

In [14]:
# print(first_image)

In [15]:
from torchvision import models
import torch.nn as nn
model = models.resnet18(pretrained=True)
# model
# Freeze all parameters in the pre-trained model
for param in model.parameters():
    param.requires_grad = True
# # Extracting features from last CNN layer
num_features = model.fc.in_features
model.fc = nn.Linear(num_features , 38)
# model

/home/.conda/envs/pytorch/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/.conda/envs/pytorch/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [17]:
# optimizer and loss function
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr = 1e-5)
loss_function = nn.CrossEntropyLoss()

In [18]:
for epoch in range(50):
    start = time.time()
    
    tr_acc = 0
    val_acc = 0
    
    # Train
    model.train()
    
    with tqdm(Train_dataloader, unit="batch") as tepoch:
        for xtrain, ytrain in tepoch:
            
            train_prob = model(xtrain)
        
            
            train_loss = loss_function(train_prob, ytrain)
            train_loss.backward() # backpropagation

            # loss.backward()
            optimizer.step() # update the model parameters
            optimizer.zero_grad()
            # training ends
            
            train_pred = torch.max(train_prob, 1).indices
            tr_acc += int(torch.sum(train_pred == ytrain))
            
        ep_tr_acc = tr_acc / LEN_TRAIN
    end = time.time()
    duration = (end - start) / 60
    
    # print(f"Epoch: {epoch}, Test_Loss: {train_loss}, Train_acc: {ep_tr_acc}")
    # Evaluate
    
    model.eval()
#     # with torch.no_grad():
    with torch.no_grad():
        for xval, yval in Val_dataloader:
            val_prob = model(xval)
            
            val_loss = loss_function(val_prob , yval)
            
            val_pred = torch.max(val_prob,1).indices
            val_acc += int(torch.sum(val_pred == yval))
            
        ep_val_acc = val_acc / LEN_val
    
    end = time.time()
    duration = (end - start) / 60
    
    print(f"Epoch: {epoch}, Test_Loss: {train_loss},Validation_loss: {val_loss}, Train_acc: {ep_tr_acc}, Val_acc: {ep_val_acc}")
    

100%|██████████| 275/275 [52:43<00:00, 11.50s/batch]


Epoch: 0, Test_Loss: 0.6949630379676819,Validation_loss: 0.576035737991333, Train_acc: 0.6622092609716196, Val_acc: 0.930286819940815


100%|██████████| 275/275 [52:11<00:00, 11.39s/batch]


Epoch: 1, Test_Loss: 0.22621706128120422,Validation_loss: 0.1537151038646698, Train_acc: 0.9567110036275696, Val_acc: 0.9691554746187115


100%|██████████| 275/275 [52:26<00:00, 11.44s/batch]


...

Epoch: 7, Test_Loss: 0.01818525791168213,Validation_loss: 0.014877483248710632, Train_acc: 0.9980510704886549, Val_acc: 0.9925449578875484


100%|██████████| 275/275 [56:57<00:00, 12.43s/batch]


Epoch: 8, Test_Loss: 0.011092079803347588,Validation_loss: 0.009810097515583038, Train_acc: 0.9986343267657728, Val_acc: 0.9938538584111086


 15%|█▌        | 42/275 [10:31<58:25, 15.05s/batch]  


KeyboardInterrupt: 